## HW4


In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from utils import ml_utils
from typing import Callable, Union, Any



In [3]:
test = pd.read_csv('data/spotify_test.csv')
train = pd.read_csv('data/spotify_train.csv')

# Part A


In [13]:
def calc_accuracy(y_pred, y_actual):
    correct_cnt = 0
    total = len(y_actual)
    #assume that y_pred and y_actual have same length

    for i in range(len(y_actual)):
        pre = y_pred[i]
        act = y_actual[i]
        if pre == act:
            correct_cnt += 1
    
    return correct_cnt/total

def n_folds(folds,train):
    for f in range(folds):
        train_fold = train[train.index % folds != f]
        valid_fold = train[train.index % folds == f]
    return train_fold, valid_fold

# def tokenize(x):
#     #tokenize the predict output to int
#     dic = {'hip-hop' : 0, 'pop': 1, 'rock': 2}
#     y = dic[x]
#     return y

# tokenized_pred = np.array(list(map(tokenize, y_pred)))
# tokenized_actual = np.array(list(map(tokenize, y_actual)))
# print(tokenized_actual)

def predict_data(data,tree):
    # given a subset dataframe as test data, predict its output alongside with its original answer
    y_actual = data['track_genre'].to_numpy()
    #apply decision tree prediction to each of the row 
    y_pred = data.apply(lambda row: tree.predict(row), axis = 1).to_numpy()

    return y_actual, y_pred

def apply_DTree(train: pd.DataFrame,
                validation: pd.DataFrame,
                test: pd.DataFrame,
                impurity_func: str,
                discrete_threshold: int = 10,
                max_depth: int = None,
                min_instances: int = 2,
                target_impurity: float = 0.0
                ):
    
    impurity_func = ml_utils.metric.entropy if impurity_func == 'entropy'   else  ml_utils.metric.gini
    tree = ml_utils.experimental.DecisionTree(discrete_threshold=discrete_threshold,
                                            max_depth=max_depth,
                                            min_instances=min_instances,
                                            target_impurity=target_impurity,
                                            impurity_func=impurity_func)
    tree.train(train,'track_genre')
    validation_accuracy = 'N/A'
    test_accuracy = 'N/A'
    if validation is not None:
        validation_accuracy = calc_accuracy(*predict_data(validation,tree))
    if test is not None:
        test_accuracy = calc_accuracy(*predict_data(test,tree))

    return validation_accuracy, test_accuracy




# Part B

In [14]:
#An example of the accuracy in 10 fold cross validation on trained modal, comparing with accuracy in test data.


train_fold, valid_fold = n_folds(10,train)
print('Training with: impurity_func: {}, max_depth: {}, min_instances: {}, target_impurity: {}'
          .format( 'entropy',5,2,0))
validation_accuracy, test_accuracy = apply_DTree(train = train_fold,
            validation = valid_fold,
            test = test,
            impurity_func = 'entropy',
            discrete_threshold = 10,
            max_depth = 10,
            min_instances = 3,
            target_impurity = 0)
print('Result: Validation accuray: {} || Test accuray: {} \n'.format(validation_accuracy,test_accuracy))


Training with: impurity_func: entropy, max_depth: 5, min_instances: 2, target_impurity: 0
Result: Validation accuray: 0.73 || Test accuray: 0.751 



# Part C

In [12]:
# records = pd.DataFrame(columns= ['impurity_func', 'max_depth', 'min_instances', 'target_impurity','validation_accuracy','test_accuracy'])
# row = {'impurity_func': 1, 'max_depth': 1, 'min_instances': 1, 'target_impurity': 1,'validation_accuracy': validation_accuracy,'test_accuracy': test_accuracy}
# row2 = {'impurity_func': 1, 'max_depth': 2, 'min_instances': 1, 'target_impurity': 1,'validation_accuracy': validation_accuracy,'test_accuracy': test_accuracy}
# records = pd.concat([records,pd.DataFrame(row,index=[0])])
# records = pd.concat([records,pd.DataFrame(row2,index=[len(records)])])
# records
print(np.arange(0,1,1))

[0]


In [11]:
train_fold, valid_fold = n_folds(10,train)
# impurity_funcs = [ml_utils.metric.entropy,ml_utils.metric.gini]
impurity_funcs = ['entorpy']
def para_tuning():
    records = pd.DataFrame(columns= ['impurity_func', 'max_depth', 'min_instances', 'target_impurity','validation_accuracy','test_accuracy'])
    for max_depth in range(2,26,2):
        for min_instances in range(3,4):
            for target_impurity in np.arange(0,1,0.6):
                for impurity_func in impurity_funcs:
                    try:
                        validation_accuracy, test_accuracy =apply_DTree(train = train_fold,
                                    validation = valid_fold,
                                    test = None,
                                    impurity_func = impurity_func,
                                    discrete_threshold = 10,
                                    max_depth = max_depth,
                                    min_instances = min_instances,
                                    target_impurity = target_impurity)
                        row = {'impurity_func': impurity_func, 'max_depth': max_depth, 'min_instances': min_instances, 'target_impurity': target_impurity,'validation_accuracy': validation_accuracy,'test_accuracy': test_accuracy}
                        records = pd.concat([records,pd.DataFrame(row,index=[len(records)])])
                    except:
                        print('Failed with: impurity_func: {}, max_depth: {}, min_instances: {}, target_impurity: {}'.format(impurity_func,max_depth,min_instances,target_impurity))
    return records

In [12]:
records =para_tuning()
records

,impurity_func,max_depth,min_instances,target_impurity,validation_accuracy,test_accuracy
0,entorpy,2,3,0.0,0.605,N/A
1,entorpy,2,3,0.6,0.510,N/A
2,entorpy,4,3,0.0,0.680,N/A
3,entorpy,4,3,0.6,0.510,N/A
4,entorpy,6,3,0.0,0.775,N/A
5,entorpy,6,3,0.6,0.510,N/A
6,entorpy,8,3,0.0,0.755,N/A
7,entorpy,8,3,0.6,0.510,N/A
8,entorpy,10,3,0.0,0.735,N/A
9,entorpy,10,3,0.6,0.510,N/A


In [ ]:
records2 = para_tuning()

In [9]:
best=records.loc[records['validation_accuracy'].idxmax()]
print(best)

impurity_func          entorpy
max_depth                   10
min_instances                4
target_impurity              0
validation_accuracy      0.735
test_accuracy              N/A
Name: 0, dtype: object


In [10]:
print('Training with: impurity_func: {}, max_depth: {}, min_instances: {}, target_impurity: {}'
          .format( 'entropy',5,2,0))
validation_accuracy, test_accuracy = apply_DTree(train = train_fold,
            validation = valid_fold,
            test = test,
            impurity_func = 'entropy',
            discrete_threshold = 10,
            max_depth = 11,
            min_instances = 4,
            target_impurity = 0)
print('Result: Validation accuray: {} || Test accuray: {} \n'.format(validation_accuracy,test_accuracy))

Training with: impurity_func: entropy, max_depth: 5, min_instances: 2, target_impurity: 0
Result: Validation accuray: 0.73 || Test accuray: 0.747 

